# Relation extraction using distant supervision: experiments

In [1]:
__author__ = "Bill MacCartney and Christopher Potts"
__version__ = "CS224u, Stanford, Spring 2020"

## Contents

1. [Overview](#Overview)
1. [Set-up](#Set-up)
1. [Building a classifier](#Building-a-classifier)
  1. [Featurizers](#Featurizers)
  1. [Experiments](#Experiments)
1. [Analysis](#Analysis)
  1. [Examining the trained models](#Examining-the-trained-models)
  1. [Discovering new relation instances](#Discovering-new-relation-instances)

## Overview

OK, it's time to get (halfway) serious. Let's apply real machine learning to train a classifier on the training data, and see how it performs on the test data. We'll begin with one of the simplest machine learning setups: a bag-of-words feature representation, and a linear model trained using logistic regression.

Just like we did in the unit on [supervised sentiment analysis](https://github.com/cgpotts/cs224u/blob/master/sst_02_hand_built_features.ipynb), we'll leverage the `sklearn` library, and we'll introduce functions for featurizing instances, training models, making predictions, and evaluating results.

## Set-up

See [the first notebook in this unit](rel_ext_01_task.ipynb#Set-up) for set-up instructions.

In [1]:
from collections import Counter
import os
import rel_ext
import utils

In [2]:
# Set all the random seeds for reproducibility. Only the
# system seed is relevant for this notebook.

utils.fix_random_seeds()

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
rel_ext_data_home = os.path.join('data', 'rel_ext_data')

With the following steps, we build up the dataset we'll use for experiments; it unites a corpus and a knowledge base in the way we described in [the previous notebook](rel_ext_01_task.ipynb).

In [4]:
corpus = rel_ext.Corpus(os.path.join(rel_ext_data_home, 'corpus.tsv.gz'))

In [5]:
kb = rel_ext.KB(os.path.join(rel_ext_data_home, 'kb.tsv.gz'))

In [6]:
dataset = rel_ext.Dataset(corpus, kb)

The following code splits up our data in a way that supports experimentation:

In [7]:
splits = dataset.build_splits()

splits

{'tiny': Corpus with 3,474 examples; KB with 445 triples,
 'train': Corpus with 249,003 examples; KB with 34,229 triples,
 'dev': Corpus with 79,219 examples; KB with 11,210 triples,
 'all': Corpus with 331,696 examples; KB with 45,884 triples}

## Building a classifier

### Featurizers

Featurizers are functions which define the feature representation for our model. The primary input to a featurizer will be the `KBTriple` for which we are generating features. But since our features will be derived from corpus examples containing the entities of the `KBTriple`, we must also pass in a reference to a `Corpus`. And in order to make it easy to combine different featurizers, we'll also pass in a feature counter to hold the results.

Here's an implementation for a very simple bag-of-words featurizer. It finds all the corpus examples containing the two entities in the `KBTriple`, breaks the phrase appearing between the two entity mentions into words, and counts the words. Note that it makes no distinction between "forward" and "reverse" examples.

In [8]:
def simple_bag_of_words_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    return feature_counter

Here's how this featurizer works on a single example:

In [9]:
kbt = kb.kb_triples[0]

kbt

KBTriple(rel='contains', sbj='Brickfields', obj='Kuala_Lumpur_Sentral_railway_station')

In [10]:
corpus.get_examples_for_entities(kbt.sbj, kbt.obj)[0].middle

'it was just a quick 10-minute walk to'

In [11]:
simple_bag_of_words_featurizer(kb.kb_triples[0], corpus, Counter())

Counter({'it': 1,
         'was': 1,
         'just': 1,
         'a': 1,
         'quick': 1,
         '10-minute': 1,
         'walk': 1,
         'to': 2,
         'the': 1})

You can experiment with adding new kinds of features just by implementing additional featurizers, following `simple_bag_of_words_featurizer` as an example.

Now, in order to apply machine learning algorithms such as those provided by `sklearn`, we need a way to convert datasets of `KBTriple`s into feature matrices. The following steps achieve that: 

In [12]:
kbts_by_rel, labels_by_rel = dataset.build_dataset()

featurized = dataset.featurize(kbts_by_rel, featurizers=[simple_bag_of_words_featurizer])

In [29]:
kbts_by_rel['capital'][-10:]

[KBTriple(rel='capital', sbj='John_Hanke', obj='Jack_Dangermond'),
 KBTriple(rel='capital', sbj='Jagapati_Babu', obj='UTV_Motion_Pictures'),
 KBTriple(rel='capital', sbj='Harry_Belafonte', obj='Samuel_L._Jackson'),
 KBTriple(rel='capital', sbj='Charles_Darwin', obj='Carl_Linnaeus'),
 KBTriple(rel='capital', sbj='Edward_Kleban', obj='Nicholas_Dante'),
 KBTriple(rel='capital', sbj='New_Zealand', obj='Robert_Muldoon'),
 KBTriple(rel='capital', sbj='Carthage', obj='Punic_Wars'),
 KBTriple(rel='capital', sbj='RZA', obj='Black_Dice'),
 KBTriple(rel='capital', sbj='Piano', obj='Bandleader'),
 KBTriple(rel='capital', sbj='Focus_Features', obj='Ian_Somerhalder')]

### Experiments

Now we need some functions to train models, make predictions, and evaluate the results. We'll start with `train_models()`. This function takes as arguments a dictionary of data splits, a list of featurizers, the name of the split on which to train, and a model factory, which is a function which initializes an `sklearn` classifier. It returns a dictionary holding the featurizers, the vectorizer that was used to generate the training matrix, and a dictionary holding the trained models, one per relation.

In [13]:
train_result = rel_ext.train_models(
    splits, 
    featurizers=[simple_bag_of_words_featurizer])

Next comes `predict()`. This function takes as arguments a dictionary of data splits, the outputs of `train_models()`, and the name of the split for which to make predictions. It returns two parallel dictionaries: one holding the predictions (grouped by relation), the other holding the true labels (again, grouped by prediction).

In [14]:
predictions, true_labels = rel_ext.predict(
    splits, train_result, split_name='dev')

Now `evaluate_predictions()`. This function takes as arguments the parallel dictionaries of predictions and true labels produced by `predict()`. It prints summary statistics for each relation, including precision, recall, and F<sub>0.5</sub>-score, and it returns the macro-averaged F<sub>0.5</sub>-score.

In [15]:
rel_ext.evaluate_predictions(predictions, true_labels)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.860      0.364      0.676        407       7057
author                    0.800      0.505      0.716        657       7307
capital                   0.653      0.254      0.497        126       6776
contains                  0.780      0.605      0.738       4487      11137
film_performance          0.794      0.568      0.736        984       7634
founders                  0.803      0.407      0.672        469       7119
genre                     0.623      0.161      0.396        205       6855
has_sibling               0.842      0.246      0.567        625       7275
has_spouse                0.864      0.345      0.664        754       7404
is_a                      0.649      0.233      0.478        618       7268
nationality               0.622      0.192      0.429        386       7036
parents     

0.5649976385557466

Finally, we introduce `rel_ext.experiment()`, which basically chains together `rel_ext.train_models()`, `rel_ext.predict()`, and `rel_ext.evaluate_predictions()`. For convenience, this function returns the output of `rel_ext.train_models()` as its result.

Running `rel_ext.experiment()` in its default configuration will give us a baseline result for machine-learned models.

In [16]:
_ = rel_ext.experiment(
    splits,
    featurizers=[simple_bag_of_words_featurizer])

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.860      0.364      0.676        407       7057
author                    0.800      0.505      0.716        657       7307
capital                   0.653      0.254      0.497        126       6776
contains                  0.780      0.605      0.738       4487      11137
film_performance          0.794      0.568      0.736        984       7634
founders                  0.803      0.407      0.672        469       7119
genre                     0.623      0.161      0.396        205       6855
has_sibling               0.842      0.246      0.567        625       7275
has_spouse                0.864      0.345      0.664        754       7404
is_a                      0.649      0.233      0.478        618       7268
nationality               0.622      0.192      0.429        386       7036
parents     

Considering how vanilla our model is, these results are quite surprisingly good! We see huge gains for every relation over our `top_k_middles_classifier` from [the previous notebook](rel_ext_01_task.ipynb#A-simple-baseline-model). This strong performance is a powerful testament to the effectiveness of even the simplest forms of machine learning.

But there is still much more we can do. To make further gains, we must not treat the model as a black box. We must open it up and get visibility into what it has learned, and more importantly, where it still falls down.

## Analysis

### Examining the trained models

One important way to gain understanding of our trained model is to inspect the model weights. What features are strong positive indicators for each relation, and what features are strong negative indicators?

In [17]:
rel_ext.examine_model_weights(train_result)

Highest and lowest feature weights for relation adjoins:

     2.520 Córdoba
     2.498 Taluks
     2.436 Valais
     ..... .....
    -1.175 century
    -1.385 he
    -1.407 who

Highest and lowest feature weights for relation author:

     2.700 books
     2.540 writer
     2.507 wrote
     ..... .....
    -2.091 famed
    -3.137 1818
    -5.516 dystopian

Highest and lowest feature weights for relation capital:

     3.431 capital
     1.988 especially
     1.911 city
     ..... .....
    -1.096 and
    -1.220 century
    -1.355 Westminster

Highest and lowest feature weights for relation contains:

     2.889 bordered
     2.083 districts
     2.081 third-largest
     ..... .....
    -2.430 film
    -2.453 Brooklyn
    -3.098 6th

Highest and lowest feature weights for relation film_performance:

     3.976 starring
     3.859 opposite
     3.402 movie
     ..... .....
    -2.054 Tamil
    -2.212 Iruvar
    -3.489 Mohabbatein

Highest and lowest feature weights for relation founders

By and large, the high-weight features for each relation are pretty intuitive — they are words that are used to express the relation in question. (The counter-intuitive results merit a bit of investigation!)

The low-weight features (that is, features with large negative weights) may be a bit harder to understand. In some cases, however, they can be interpreted as features which indicate some _other_ relation which is anti-correlated with the target relation. (As an example, "directed" is a negative indicator for the `author` relation.)

__Optional exercise:__ Investigate one of the counter-intuitive high-weight features. Find the training examples which caused the feature to be included. Given the training data, does it make sense that this feature is a good predictor for the target relation?

<!--
- SPOILER: Using `penalty='l1'` results in somewhat less intuitive feature weights, and about the same performance.
- SPOILER: Using `penalty='l1', C=0.1` results in much more intuitive feature weights, but much worse performance.
-->

### Discovering new relation instances

Another way to gain insight into our trained models is to use them to discover new relation instances that don't currently appear in the KB. In fact, this is the whole point of building a relation extraction system: to extend an existing KB (or build a new one) using knowledge extracted from natural language text at scale. Can the models we've trained do this effectively?

Because the goal is to discover new relation instances which are _true_ but _absent from the KB_, we can't evaluate this capability automatically. But we can generate candidate KB triples and manually evaluate them for correctness.

To do this, we'll start from corpus examples containing pairs of entities which do not belong to any relation in the KB (earlier, we described these as "negative examples"). We'll then apply our trained models to each pair of entities, and sort the results by probability assigned by the model, in order to find the most likely new instances for each relation.

In [18]:
rel_ext.find_new_relation_instances(
    dataset,
    featurizers=[simple_bag_of_words_featurizer])

Highest probability examples for relation adjoins:

     1.000 KBTriple(rel='adjoins', sbj='Canada', obj='Vancouver')
     1.000 KBTriple(rel='adjoins', sbj='Vancouver', obj='Canada')
     1.000 KBTriple(rel='adjoins', sbj='Mexico', obj='Atlantic_Ocean')
     1.000 KBTriple(rel='adjoins', sbj='Atlantic_Ocean', obj='Mexico')
     1.000 KBTriple(rel='adjoins', sbj='Pakistan', obj='Lahore')
     1.000 KBTriple(rel='adjoins', sbj='Lahore', obj='Pakistan')
     1.000 KBTriple(rel='adjoins', sbj='Sicily', obj='Italy')
     1.000 KBTriple(rel='adjoins', sbj='Italy', obj='Sicily')
     1.000 KBTriple(rel='adjoins', sbj='Italy', obj='Palermo')
     1.000 KBTriple(rel='adjoins', sbj='Palermo', obj='Italy')

Highest probability examples for relation author:

     1.000 KBTriple(rel='author', sbj='The_Doors_of_Perception', obj='Aldous_Huxley')
     1.000 KBTriple(rel='author', sbj='Charles_Dickens', obj='A_Christmas_Carol')
     1.000 KBTriple(rel='author', sbj='Aldous_Huxley', obj='The_Doors_of_P

     1.000 KBTriple(rel='parents', sbj='Philip_II_of_Macedon', obj='Alexander_the_Great')
     1.000 KBTriple(rel='parents', sbj='Gutzon_Borglum', obj='Lincoln_Borglum')
     1.000 KBTriple(rel='parents', sbj='Alexander_the_Great', obj='Philip_II_of_Macedon')
     1.000 KBTriple(rel='parents', sbj='Lincoln_Borglum', obj='Gutzon_Borglum')
     1.000 KBTriple(rel='parents', sbj='Thomas_Boleyn,_1st_Earl_of_Wiltshire', obj='Anne_Boleyn')
     1.000 KBTriple(rel='parents', sbj='Anne_Boleyn', obj='Thomas_Boleyn,_1st_Earl_of_Wiltshire')
     1.000 KBTriple(rel='parents', sbj='Jess_Margera', obj='April_Margera')
     1.000 KBTriple(rel='parents', sbj='April_Margera', obj='Jess_Margera')
     1.000 KBTriple(rel='parents', sbj='Kaho_Naa..._Pyaar_Hai', obj='Hrithik_Roshan')
     1.000 KBTriple(rel='parents', sbj='Hrithik_Roshan', obj='Kaho_Naa..._Pyaar_Hai')

Highest probability examples for relation place_of_birth:

     0.999 KBTriple(rel='place_of_birth', sbj='Uttar_Pradesh', obj='Lucknow')
  

There are actually some good discoveries here! The predictions for the `author` relation seem especially good. Of course, there are also plenty of bad results, and a few that are downright comical. We may hope that as we improve our models and optimize performance in our automatic evaluations, the results we observe in this manual evaluation improve as well.

__Optional exercise:__ Note that every time we predict that a given relation holds between entities `X` and `Y`, we also predict, with equal confidence, that it holds between `Y` and `X`. Why? How could we fix this?

\[ [top](#Relation-extraction-using-distant-supervision) \]